In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import datetime

In [ ]:
items= pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
shops= pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
sales_train= pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
test= pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
sample_submission= pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")
item_categories= pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")

In [ ]:
shops.head()

In [ ]:
sales_train.head()

In [ ]:
shops.loc[shops["shop_name"] == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'

In [ ]:
shops["city"]=shops["shop_name"].str.split(' ').map(lambda x: x[0])

In [ ]:
shops["name_of_shop"] = shops["shop_name"].str.split(' ').map(lambda x: x[1])

In [ ]:
shops.loc[shops["city"] =="!Якутск","city"] = "Якутск"


In [ ]:

shops.loc[shops["name_of_shop"] =='"Распродажа"',"name_of_shop"] = "other"
shops.loc[shops["name_of_shop"] =="(Плехановская,","name_of_shop"] = "other"
shops.loc[shops["name_of_shop"] =="МТРЦ","name_of_shop"] = "ТРЦ"

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
shops["name_of_shop"] = LabelEncoder().fit_transform(shops["name_of_shop"])
shops["city"] = LabelEncoder().fit_transform(shops["city"])

In [ ]:
shops

dale dobro ...staro


In [ ]:
sales_train.info()

In [ ]:
test.info()

In [ ]:
sales_train = sales_train.merge(items, on = "item_id" )

In [ ]:
#sales_train = sales_train.merge(shops, on = "shop_id" )

In [ ]:
sales_train.head()

In [ ]:
sales_train = sales_train.drop(["item_name","item_category_id","item_price"], axis = 1)

In [ ]:
sales_train["date"] = pd.to_datetime(sales_train["date"])

In [ ]:
sales_train.head()

In [ ]:
sales_train["date_block_num"].unique()

In [ ]:
sales_train_monthly =pd.DataFrame(sales_train.sort_values("date").groupby(["date_block_num","shop_id","item_id"])["item_cnt_day"].agg('sum').reset_index())

In [ ]:
sales_train_monthly.describe().T

In [ ]:
sales_train_monthly_pivot = sales_train_monthly.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num']).reset_index()

In [ ]:
sales_train_monthly_pivot

In [ ]:
sales_train_monthly_pivot= pd.merge(test,sales_train_monthly_pivot,on = ['item_id','shop_id'],how = 'left')

In [ ]:
sales_train_monthly_pivot.fillna(0,inplace = True)

In [ ]:
sales_train_monthly_pivot.head(10)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#sales_train_monthly_pivot = sales_train_monthly_pivot.drop(["shop_id", "item_id", "ID"], axis = 1)


In [ ]:
sales_train_monthly_pivot.head()

In [ ]:
#proba
X= sales_train_monthly_pivot.iloc[:,:-1]
y= sales_train_monthly_pivot.iloc[:,-1]

In [ ]:
y =y.values.reshape(-1,1)

In [ ]:
print(X.shape, y.shape)

In [ ]:
#proba2
X_train= sales_train_monthly_pivot.iloc[:,:-1]
y_train= sales_train_monthly_pivot.iloc[:,-1]
X_test = sales_train_monthly_pivot.iloc[:,1:]


In [ ]:
y_train =y_train.values.reshape(-1,1)

In [ ]:
print(X_train.shape,y_train.shape,X_test.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_train.info()

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)


In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

In [ ]:
print(X_test.shape, X_test.shape)

# 

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout

In [ ]:
my_model = Sequential()
my_model.add(LSTM(units = 10,return_sequences = True,input_shape = (36,1)))
my_model.add(Dropout(0.2))
my_model.add(LSTM(units = 10,return_sequences = True))
my_model.add(Dropout(0.2))
my_model.add(LSTM(units = 10,return_sequences = True))
my_model.add(Dropout(0.2))
my_model.add(LSTM(units = 10,return_sequences = True))
my_model.add(Dropout(0.2))
my_model.add(LSTM(units = 10,return_sequences = True))
my_model.add(Dropout(0.2))
my_model.add(LSTM(units = 10,return_sequences = False))
my_model.add(Dropout(0.2))
my_model.add(Dense(1))




my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()

In [ ]:
my_model.fit(X_train,y_train,batch_size = 12500,epochs = 20)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
losses = pd.DataFrame(my_model.history.history)

In [ ]:
losses.plot()

In [ ]:
"""#test podatke vzemen
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))"""

In [ ]:
X_test.shape

In [ ]:
submission_pfs = my_model.predict(X_test)

In [ ]:
submission_pfs = submission_pfs.clip(0,20)


In [ ]:
submission = pd.DataFrame({'ID':test['ID'],'item_cnt_month':submission_pfs.ravel()})

In [ ]:
submission.to_csv('forceste_price_36.csv', index = False)